# Open COVID-19 Analysis 

This notebook is meant to serve as a starter for anyone interested in diving into the most recent data surrounding the global 2019 novel coronavirus (COVID-19) outbreak.

Datasets:
- Epidemiology Data: https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases
- Population Data: https://population.un.org/wpp/Download/Standard/Population/

In [1]:
import pandas as pd
import cufflinks as cf
import plotly.graph_objs as go
import numpy as np
from scipy.optimize import curve_fit
from typing import List
cf.go_offline()

In [2]:
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

In [3]:
raw_confirmed = pd.read_csv(confirmed_url)
raw_deaths = pd.read_csv(deaths_url)
raw_recovered = pd.read_csv(recovered_url)

In [4]:
total_pop = pd.read_csv("total_pop.csv")
total_pop.index = total_pop["Country"]
total_pop.drop(axis=1, labels=["Country"], inplace=True)
def current_pop(country: str):
    return int(total_pop.T[country]["2020"].replace(" ", "")) * 1000

In [5]:
def country_agg(raw_df):
    df = raw_df.copy()
    df.drop(axis=1, labels=["Province/State", "Lat", "Long"], inplace=True)
    df = df.groupby(by="Country/Region").sum()
    df.drop(axis=0, labels=["Others"], inplace=True) # Drop cases not associated with a country
    return df

In [6]:
confirmed = country_agg(raw_confirmed)
deaths = country_agg(raw_deaths)
recovered = country_agg(raw_recovered)
infected = confirmed - recovered - deaths

In [7]:
# Top countries by number of confirmed cases
confirmed.T.max().sort_values(ascending=False).head(20)

Country/Region
Mainland China    80699
Italy              7375
South Korea        7314
Iran               6566
France             1126
Germany            1040
Spain               673
US                  537
Japan               502
Switzerland         337
UK                  273
Netherlands         265
Sweden              203
Belgium             200
Norway              176
Singapore           150
Hong Kong           114
Austria             104
Malaysia             99
Bahrain              85
dtype: int64

In [8]:
def sird_plot(country_name: str, pop_prop: bool = True):
    pop_size = current_pop(country_name)/100000 if pop_prop else 1
    plot_data = pd.DataFrame([infected.T[country_name]/pop_size, recovered.T[country_name]/pop_size, deaths.T[country_name]/pop_size], index=["Infected", "Recovered", "Dead"]).T
    if(pop_prop):
        plot_data.iplot(title="COVID-19 Outbreak: " + country_name, yTitle="Count per 100,000 People")
    else:
        plot_data.iplot(title="COVID-19 Outbreak: " + country_name,  yTitle="Count")
def country_plot(cnames: List[str], pop_prop: bool = True):
    col_data = [];
    for c in cnames:
        pop_size = current_pop(c)/100000 if pop_prop else 1
        col_data.append(confirmed.T[c]/pop_size);
    plot_data = pd.DataFrame(col_data, index=cnames).T
    if(pop_prop):
        plot_data.iplot(subplots=True, subplot_titles=True, shared_yaxis=True, title="COVID-19 Confirmed Cases Per 100,000 People")
    else:
        plot_data.iplot(subplots=True, subplot_titles=True, shared_yaxis=True, title="COVID-19 Confirmed Cases")

In [9]:
top_countries = list(confirmed.T.sum().sort_values(ascending=False).head(4).index)
top_countries.append("US")
country_plot(top_countries, pop_prop=False)
country_plot(top_countries, pop_prop=True)

In [10]:
for c in list(confirmed.T.sum().sort_values(ascending=False).head(10).index):
    sird_plot(c, pop_prop=False)

# Regression Analysis

In [11]:
# Removes rows prior to onset of outbreak and sets index to days since outbreak began in country
def confirmed_by_days(country_name: str):
    c_series = confirmed.T[country_name]
    c_series = c_series[c_series.cumsum() > 0]
    return c_series.reset_index(drop=True)
def exp_reg(country_name: str):
    def exp_fn(x, a, b):
        return a * np.exp(b * x)
    reg_data = 100000 * confirmed_by_days(country_name) / current_pop(country_name)
    try:
        popt, pcov = curve_fit(exp_fn, reg_data.index.values, reg_data.values, bounds=(0, [np.inf, np.inf]))
        pred_data = exp_fn(range(len(reg_data.index) + 5), *popt)
        formula_str = "%0.2e*exp(%0.2f*x)" % (popt[0], popt[1])
        pd.DataFrame([pred_data, reg_data], index=[formula_str, "Actual"]).T.iplot(
            title="Forecasting COVID-19 Confirmed Cases: " + country_name,
            xTitle="Days since first confirmed case in country",
            yTitle="# of confirmed cases per 100,000 people",
            legend="bottom"
        )
    except RuntimeError:
        print("Failed to generate adequate regression for: " + country_name)

In [12]:
for c in list(confirmed.T.sum().sort_values(ascending=False).head(10).index):
    exp_reg(c)